In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm
import re
import time
import pickle

pd.set_option('display.max_colwidth', 200)
#import_libraries_elmo.py hosted with GitHub

In [ ]:
#read data
train = pd.read_csv("train_2kmZucJ.csv")
test = pd.read_csv("test_oJQbWVk.csv")

train.shape, test.shape

In [ ]:
train['label'].value_counts(normalize = True)

In [ ]:
train.head()

now we are going to pre-process the input data before feeding it to the model.

In [ ]:
#first, we must remove the URL backeground from the input data(tweets).
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))


In [ ]:
#now we have to clean the text.
#remove punctuation marks.
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'
train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

#then converting the text to the lowercase.
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

#next we have to remove the numbers
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

#remove white spaces 
train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

Now we have clean the input data by removing vast amount of unneccasry data.
Then it is time to import spacy's langauge model

In [ ]:
#impoting the language model
nlp = spacy.load('en', disable=['parser', 'ner'])

#function to normalize/lemmitize the text
def lemmitization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [ ]:
#checking a training sample from the dataset:
train.sample(10)

We need tensoflow to build the ELMo model 
we have alrady installed and imported the tensorflow package. 
But now we have to install similar library called tensorflow-hub.
to install, run following command:
pip install tensorflow-hub

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

#creating the elmo model
elmo =  hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
#random coding sentence:
x = ["Roasted ants are a popular snack in Columbia"]

#extracting the elmo features:
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
embeddings.shape 

In [ ]:
#now lets identify the vectors for our elmo model:
#(every word in the input sentence has an ELMo vector of size 1024).

def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))
 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg =  LogisticRegression()
lreg.fit(xtrain, ytrain) 

preds_valid = lreg.predict(xvalid)

In [ ]:
# make predictions on test set
preds_test = lreg.predict(elmo_test_new)

#prepare submission dataframe
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})
# write predictions to a CSV file
sub.to_csv("sub_lreg.csv", index=False)